In [ ]:
# DATA ENGINEER EXERCISE 1

# Import pandas
import pandas as pd

# Load Data
df_info = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv")
df_email = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv")
df_status = pd.read_csv("https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv")


In [ ]:
# Verify data loaded
df_info.head()

In [ ]:
df_email.head()

In [ ]:
df_status.head()

In [ ]:
# Explore data
df_info.info()

In [ ]:
df_email.info()

In [ ]:
df_status.info()

In [ ]:
# Modify df_info (only need certain columns)
df_info2 = df_info[["cons_id","source","create_dt","modified_dt"]]
df_info2.info()

In [ ]:
# Modify df_email (only need primary email and certain columns)
df_email2 = df_email[df_email["is_primary"]==1]
df_email2 = df_email2[["cons_email_id","cons_id","email"]]
df_email2.info()

In [ ]:
# Modify df_status (only need chapter=1 and certain columns)
df_status2 = df_status[df_status["chapter_id"]==1]
df_status2 = df_status2[["cons_email_id","isunsub"]]
df_status2.info()

In [ ]:
# For all primary emails, merge code, created_dt, and updated_dt from df_info)
df_infoemail = pd.merge(df_email2,df_info2, on='cons_id', how='left')
df_infoemail.info()

In [ ]:
# For all primary emails, merge isunsub from df_status)
df_infoemailstatus = pd.merge(df_infoemail,df_status2,on='cons_email_id',how='left')
df_infoemailstatus.info()

In [ ]:
# If email unavailable, assume not unsubscribed
df_infoemailstatus["isunsub"]=df_infoemailstatus["isunsub"].fillna(0)
df_infoemailstatus["isunsub"].value_counts()

In [ ]:
# Select only needed columns and rename
df_infoemailstatus=df_infoemailstatus.rename(columns={"source": "code", "isunsub": "is_unsub", "create_dt":"created_dt", "modified_dt":"updated_dt"})
df_infoemailstatus=df_infoemailstatus.drop(columns=['cons_email_id', 'cons_id'])
df_infoemailstatus.info()

In [ ]:
# Change datetypes
df_infoemailstatus['created_dt']= pd.to_datetime(df_infoemailstatus['created_dt'])

In [ ]:
df_infoemailstatus['updated_dt']= pd.to_datetime(df_infoemailstatus['updated_dt'])

In [ ]:
df_infoemailstatus = df_infoemailstatus.astype({"is_unsub":bool})

In [ ]:
df_infoemailstatus.info()

In [ ]:
df_infoemailstatus.head()

In [ ]:
# Write to csv
df_infoemailstatus.to_csv ('people.csv', index = False, header=True)

In [ ]:
# DATA ENGINEER EXERCISE 2

# Import pandas
import pandas

# Load data file
df = pd.read_csv('people.csv')
df.info()

In [ ]:
# change created_dt datetype to datetime
df['created_dt']= pd.to_datetime(df_infoemailstatus['created_dt'])


In [ ]:
# Group by created_date, count distinct emails
df2 = df.groupby(df['created_dt'].dt.date)['email'].nunique()

In [ ]:
# Convert to dataframe
df2=pd.DataFrame(df2)

In [ ]:
df2=df2.reset_index()

In [ ]:
df2.info()

In [ ]:
# Rename columns and change datetypes
df2=df2.rename(columns={"created_dt": "acquisition_date", "email": "acquisitions"})

In [ ]:
df2['acquisition_date']= pd.to_datetime(df2['acquisition_date'])

In [ ]:
df2.info()

In [ ]:
# Write to csv
df2.to_csv ('acquisition_facts.csv', index = False, header=True)